In [ ]:
-- CREATE OR REPLACE STORAGE INTEGRATION s3_integration
--   TYPE = EXTERNAL_STAGE
--   STORAGE_PROVIDER = 'S3'
--   ENABLED = TRUE
--   STORAGE_AWS_ROLE_ARN = 'arn:aws:iam::774305574730:role/snowflake_access_role'
--   STORAGE_ALLOWED_LOCATIONS = ('s3://hanifidemirel/')

In [ ]:
desc integration s3_integration

In [ ]:
CREATE OR REPLACE FILE FORMAT PARQUET_FORMAT
   TYPE = PARQUET
   COMPRESSION = SNAPPY;
   
CREATE or REPLACE STAGE s3_stage_loan_tape
  STORAGE_INTEGRATION = s3_integration
  URL = 's3://hanifidemirel/loan_tape/'
  FILE_FORMAT = PARQUET_FORMAT;
  
CREATE or REPLACE STAGE s3_stage_pre_sale_loan_tape
  STORAGE_INTEGRATION = s3_integration
  URL = 's3://hanifidemirel/pre_sale_loan_tape/'
  FILE_FORMAT = PARQUET_FORMAT;

In [ ]:
list @s3_stage_pre_sale_loan_tape

In [ ]:
import time
from snowflake.snowpark import Session

with Session.builder.getOrCreate() as session:
        
    df = session.read.option("compression", "gzip").options({"infer_schema":True}).csv('@s3_stage_pre_sale_loan_tape')
    row_count = df.count()
    print(f"The DataFrame has {row_count} rows.")
    df.write.mode("overwrite").save_as_table("pre_sale_loan_tape")

In [ ]:
import time
from snowflake.snowpark import Session

with Session.builder.getOrCreate() as session:
        
    df = session.read.option("compression", "snappy").options({"infer_schema":True}).parquet('@s3_stage_loan_tape/part')
    row_count = df.count()
    print(f"The DataFrame has {row_count} rows.")
    df.write.mode("overwrite").save_as_table("loan_tape")

In [ ]:
SELECT * FROM hol_db.analytics.LOAN_TAPE AT(TIMESTAMP => '2024-12-18 02:58:51-08:00');


In [ ]:
update hol_db.analytics.LOAN_TAPE 
set "apr" = 5 
where "charge_ari" = 'OBV5-JVJS';
commit;

In [ ]:
SHOW TABLES HISTORY LIKE 'LOAN_TAPE%' IN hol_db.analytics;
